In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from lightonml.projections.sklearn import OPUMap

In [ ]:
def make_x1_x2(n):
    x1 = np.random.choice([0,1], size=n**2, p=[0.75,0.25]) # binary signal
    x2 = np.random.choice([0,1], size=n**2, p=[0.5,0.5]) # binary signal
    
    x2[x1>0] = 0 # this is so that after addition, x1+x2 is stil binary
    
    xs = np.vstack((x1,x2, x1+x2)) # add x1+x2
    
    return xs
    
def make_anchors(n, xs, number_of_anchors):
    anchors = np.zeros([number_of_anchors, n**2]) # store the anchors here
    
    anchor_p = [0.8,0.2]
    anchors[0] = np.random.choice([0,1], size=n**2, p=anchor_p) + xs[2] # first anchor is random binary
    
    for i in range(1, number_of_anchors):
        # add random binary to previous anchor to get next anchor
        anchors[i] = np.random.choice([0,1], size=n**2, p=anchor_p) + anchors[i-1]

    anchors[anchors>0] = 1 # input cannot be above 1
    anchors = anchors[::-1] # reverse the order for later convenience when interfering with signal
    
    return anchors

def interfere_with_anchors(n, x, anchors):
    interfered = anchors - x # interfere all anchors with signal
    interfered = np.vstack((interfered, x)) # x with zero (zero is less than x so subtract the other way)
    
    anchors = np.vstack((anchors, np.zeros(n**2))) # zero is an anchor too
    
    for i in range(anchors.shape[0]-1):
        # subtract all anchors from each other
        diffs = anchors[i] - anchors[1+i:]
        interfered = np.vstack((interfered, diffs))
    
    return interfered

In [ ]:
image_size = 64
number_of_anchors = 15
num_of_rows_in_A = 100

xs = make_x1_x2(image_size)
anchors = make_anchors(image_size, xs, number_of_anchors)

x1_inter = interfere_with_anchors(image_size, xs[0], anchors)
x2_inter = interfere_with_anchors(image_size, xs[1], anchors)
x1_plus_x2_inter = interfere_with_anchors(image_size, xs[2], anchors)

# we can put everything into the OPU in one go by concatenating
opu_input = np.vstack((x1_inter, x2_inter, x1_plus_x2_inter))

In [ ]:
def get_OPU_measurements(opu_input, num_rand_proj):
    mapping = OPUMap(n_components=num_rand_proj, verbose_level=2)
    mapping.opu.device.exposure_us = 400 # exposure needs to be chosen so that there is no saturation.
    y = mapping.transform(opu_input.astype('uint8'))
    print ('Max value:', np.max(y))
    print ('Min value: ', np.min(y))
    
    return y

# get the OPU measurements
y_quant = get_OPU_measurements(opu_input, num_of_rows_in_A)

In [ ]:
def make_D_ensembles(y, number_of_anchors):
    # populates the distance matrices for all rows
    
    num_elements = int((number_of_anchors+2)* (number_of_anchors+1) * 0.5)
    
    trials = y.shape[1]
    dim = number_of_anchors+2
    all_D_oracles_x1 = np.zeros([trials, dim, dim])
    all_D_oracles_x2 = np.zeros([trials, dim, dim])
    all_D_oracles_x1_plus_x2 = np.zeros([trials, dim, dim])
    
    ind = np.triu_indices(all_D_oracles_x1[0].shape[0], k=1)
    for i in range(trials):
        # for x1
        data = y[0:num_elements,i]
        all_D_oracles_x1[i][ind] = data
        all_D_oracles_x1[i] += all_D_oracles_x1[i].T
        
        # for x2
        data = y[num_elements: 2*num_elements,i]
        all_D_oracles_x2[i][ind] = data
        all_D_oracles_x2[i] += all_D_oracles_x2[i].T
        
        # for x3
        data = y[2*num_elements: 3*num_elements,i]
        all_D_oracles_x1_plus_x2[i][ind] = data
        all_D_oracles_x1_plus_x2[i] += all_D_oracles_x1_plus_x2[i].T
        
    return all_D_oracles_x1, all_D_oracles_x2, all_D_oracles_x1_plus_x2

def do_MDS(D, number_of_anchors):
    # does MDS
    
    m = number_of_anchors
    J = np.eye(m + 2) - 1. / (m + 2) * np.ones((m + 2, m + 2))
    G = -1/2 * np.dot(J, D).dot(J)
    U, s, VT = np.linalg.svd(G)
    Z_est_R2 = np.dot(np.diag(np.sqrt(s[:2])), VT[:2, :])
    Z_est_cpx = Z_est_R2[0, :] + 1j*Z_est_R2[1, :]
    
    # translate the origin back at (0, 0)
    Z_est_cpx -= Z_est_cpx[m + 1]
    
    return Z_est_cpx

def ortho_procrustes(fixed, modify):
    # does the Procrustes analysis following procedure Dokmanic et al. in references
    # and https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    # assumes that the points to align are in column with index 1 onwards.
    # we want to align modify with fixed
    
    fixed = np.vstack ((np.real(fixed[1:]), np.imag(fixed[1:])))
    modify = np.vstack ((np.real(modify), np.imag(modify)))
    original = modify.copy()
    modify = modify[:,1:]
    fixed_mean = (np.mean(fixed, axis=1)).reshape([-1,1])
    fixed -= fixed_mean
    modify_mean = (np.mean(modify, axis=1)).reshape([-1,1])
    modify -= modify_mean
    M = fixed @ modify.T
    u, s, vh = np.linalg.svd(M)
    R = u @ vh
    original = R @ (original - modify_mean @ np.ones([1, original.shape[1]])) + fixed_mean@np.ones([1, original.shape[1]])
    return original[0] + 1j*original[1]

In [ ]:
def make_LXX(X):
    # intermediate function for MDS with gradient descent
    
    e = np.ones([X.shape[1],1])
    G = X.T @ X
    diag_vec = np.diag(G).reshape([-1,1])
    L =  diag_vec @ e.T + e @ diag_vec.T - 2*G
    return L
    
def gradient_descent_X(D, X_0, W):
    # does gradient descent on MDS solution
    
    lr = 0.001

    n_iter = 20
    
    N = X_0.shape[1]
    e = np.ones([N,1])
    
    X = X_0.copy()
    
    for i in range(n_iter):
        L = make_LXX(X)
        P = D - L
        P = W*P
        grad = (1/N**2) * (8 * X @ (P - np.diag(np.diag(P @ e)) ))
        X -= lr*grad

    return X, L

In [ ]:
# put all the measurements into distance matrices
all_D_quant_x1, all_D_quant_x2, all_D_quant_x1_plus_x2 = make_D_ensembles(y_quant, number_of_anchors)
    
# for storing the results    
manual = np.zeros([num_of_rows_in_A, number_of_anchors+1-2]).astype('complex128')
direct = np.zeros([num_of_rows_in_A, number_of_anchors+1-2]).astype('complex128')
manual_gd = np.zeros([num_of_rows_in_A, number_of_anchors+1-2]).astype('complex128')
direct_gd = np.zeros([num_of_rows_in_A, number_of_anchors+1-2]).astype('complex128')

floor = np.min(y_quant)

for trial in tqdm(range(num_of_rows_in_A)):
    for i in range(2,number_of_anchors+1):
        # choose a random subset of the anchors
        ind = np.random.choice(np.arange(1, number_of_anchors+1), i, replace=False)
        ind = np.hstack((ind,0,number_of_anchors+1)) # we need the original signal measurement and origin
        ind.sort()

        # get the distance matrices
        D_quant_x1 = all_D_quant_x1[:,:,ind][trial][ind,:]
        D_quant_x2 = all_D_quant_x2[:,:,ind][trial][ind,:]
        D_quant_x1_plus_x2 = all_D_quant_x1_plus_x2[:,:,ind][trial][ind,:]
        
        D_quant_x1[D_quant_x1<=floor] = 0
        D_quant_x2[D_quant_x2<=floor] = 0
        D_quant_x1_plus_x2[D_quant_x1_plus_x2<=floor] = 0

        # recover points via normal MDS
        recovered_points_x1 = do_MDS(D_quant_x1, i)
        recovered_points_x2 = do_MDS(D_quant_x2, i)
        recovered_points_x1_plus_x2 = do_MDS(D_quant_x1_plus_x2, i)
        # align x2 anchors with x1 anchors
        recovered_points_x2 = ortho_procrustes(recovered_points_x1, recovered_points_x2)
        # align x1+x2 anchors with x1 anchors
        recovered_points_x1_plus_x2 = ortho_procrustes(recovered_points_x1, recovered_points_x1_plus_x2)

        # save results
        manual[trial, i-2] = recovered_points_x1[0] + recovered_points_x2[0] # recovered x1+x2
        direct[trial, i-2] = recovered_points_x1_plus_x2[0] # measured x1+x2

        # grad descent
        # get the distance matrices
        D_quant_x1 = all_D_quant_x1[:,:,ind][trial][ind,:]
        D_quant_x2 = all_D_quant_x2[:,:,ind][trial][ind,:]
        D_quant_x1_plus_x2 = all_D_quant_x1_plus_x2[:,:,ind][trial][ind,:]
        
        # for x1
        X_0 = np.vstack((np.real(recovered_points_x1), np.imag(recovered_points_x1))) # get initialization for grad descent
        W = (D_quant_x1>floor).astype('float') + np.eye(D_quant_x1.shape[0]) # grad descent mask
        X, _ = gradient_descent_X(D_quant_x1, X_0, W) # do gradient descent
        recovered_points_x1 = X[0] + 1j*X[1] # convert back to complex points
        recovered_points_x1 -= recovered_points_x1[-1] # center

        # same as above but for x2
        X_0 = np.vstack((np.real(recovered_points_x2), np.imag(recovered_points_x2)))
        W = (D_quant_x2>floor).astype('float') + np.eye(D_quant_x2.shape[0])
        X, L = gradient_descent_X(D_quant_x2, X_0, W)
        recovered_points_x2 = X[0] + 1j*X[1]
        recovered_points_x2 -= recovered_points_x2[-1]

        # same as above but for x1+x2
        X_0 = np.vstack((np.real(recovered_points_x1_plus_x2), np.imag(recovered_points_x1_plus_x2)))
        W = (D_quant_x1_plus_x2>floor).astype('float') + np.eye(D_quant_x1_plus_x2.shape[0])
        X, L = gradient_descent_X(D_quant_x1_plus_x2, X_0, W)
        recovered_points_x1_plus_x2 = X[0] + 1j*X[1]
        recovered_points_x1_plus_x2 -= recovered_points_x1_plus_x2[-1]

        # align x2 anchors with x1 anchors
        recovered_points_x2 = ortho_procrustes(recovered_points_x1, recovered_points_x2)
        # align x1+x2 anchors with x1 anchors
        recovered_points_x1_plus_x2 = ortho_procrustes(recovered_points_x1, recovered_points_x1_plus_x2)

        # save results
        manual_gd[trial, i-2] = recovered_points_x1[0] + recovered_points_x2[0]
        direct_gd[trial, i-2] = recovered_points_x1_plus_x2[0]

In [ ]:
# prevent dividing from small values which have undue influence
mask = (np.abs(direct)>2.5).astype('int')
mask_sum = (np.sum(mask, axis=0))

# prevent dividing from small values which have undue influence
mask_gd = (np.abs(direct_gd)>2.5).astype('int')
mask_sum_gd = (np.sum(mask_gd, axis=0))

# compute all relative errors
rel_errors = 100*(np.abs(manual - direct) / np.abs(direct))
rel_errors_gd = 100*(np.abs(manual_gd - direct_gd) / np.abs(direct_gd))

# compute average relative error for ones which do not have undue influence
rel_errors = np.sum(rel_errors*mask, axis=0) / mask_sum
rel_errors_gd = np.sum(rel_errors_gd*mask_gd, axis=0) /mask_sum_gd

# plot results
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(5,4))
anchors_for_plot = np.arange(2, number_of_anchors+1)
plt.plot(anchors_for_plot, rel_errors, label='MDS', linewidth=2.5)
plt.plot(anchors_for_plot, rel_errors_gd, label='MDS-GD', color='r', linestyle= (0, (5, 1)), linewidth=2.5)
plt.ylabel('Average relative error (%)')
plt.xlabel('Number of anchors')
plt.xticks(np.arange(min(anchors_for_plot), max(anchors_for_plot)+1, 2.0))
plt.grid(which='major')
plt.legend()
plt.tight_layout()